In [1]:
import numpy as np
import cv2 as cv

In [2]:
capture = cv.imread('data/temp/scr.png', cv.IMREAD_GRAYSCALE)
ns = [cv.imread('data/numbers/n%s.png' % x, cv.IMREAD_GRAYSCALE) for x in range(1, 10)]
ress = [cv.matchTemplate(capture, n, cv.TM_CCOEFF_NORMED) for n in ns]
threshold = 0.95
locs = [np.where(res >= threshold)[::-1] for res in ress]
del capture, ns, ress, threshold

In [3]:
board = np.zeros((9, 9), dtype=np.int0)
for n, loc in enumerate(locs):
    for pos in zip(*loc):
        x = pos[0] // 80
        y = (pos[1] - 256) // 80
        board[y, x] = n + 1

In [4]:
def isValidState(board:np.ndarray, cell:list) -> bool:
    x, y = cell
    cellValue = board[y, x]
    
    row = np.where(board[y, :] == cellValue)
    if len(row[0]) > 1: return False
    
    column = np.where(board[:, x] == cellValue)
    if len(column[0]) > 1: return False
    
    block = np.where(board[y // 3 * 3:y // 3 * 3 + 3, x // 3 * 3:x // 3 * 3 + 3] == cellValue)
    if len(block[0]) > 1: return False

    return True

In [5]:
def testBoard(tBoard:np.ndarray, permIndex:int) -> bool:
    '''
    Solves the sudoku on a numpy array
    Returns: bool used in the process. It will be True or False
    depending if the board is solved or not.
    '''
    x, y = permutesLoc[permIndex]
    # Go for each valid number and get deeper while its posible
    for v, val in enumerate(permutesValids[permIndex]):
        # Set the position as val
        tBoard[y, x] = val

        # If its not a valid state, skip it
        # 1* Note the "return False" at the end if no option is possible
        if not isValidState(tBoard, (x, y)):
            continue

        ## (All of this code from now on is executed only if the valid was correct)
        # If we are at the end of the chain, return True
        if permIndex == len(permutesValids) - 1:
            # print(tBoard)
            return True
        
        # If not, the recursion keeps going
        done = testBoard(tBoard, permIndex + 1)
        
        # If that recursion was done, return True
        if done:
            return True
        # If not, continue
        else:
            continue
    # If nothing was successful, undo the board and return False
    tBoard[y, x] = 0
    # 1*
    return False
        

In [6]:
permutesLoc = [[y, x] for x, y in zip(*np.where(board == 0))]
permutesValids = []
for x, y in permutesLoc:
    valids = [x for x in range(1, 10)]
    # Row check
    for n in board[y, :]:
        if n in valids:
            valids.remove(n)
    # Column check
    for n in board[:, x]:
        if n in valids:
            valids.remove(n)
    # Block check
    block = board[y // 3 * 3:y // 3 * 3 + 3, x // 3 * 3:x // 3 * 3 + 3]
    for n in block.flatten():
        if n in valids:
            valids.remove(n)
    permutesValids.append(valids)

In [7]:
tBoard = np.array(board)
done = testBoard(tBoard, 0)
tBoard

array([[9, 7, 8, 5, 4, 1, 2, 3, 6],
       [1, 3, 2, 8, 7, 6, 5, 4, 9],
       [4, 5, 6, 9, 2, 3, 7, 8, 1],
       [5, 1, 9, 2, 8, 7, 4, 6, 3],
       [7, 6, 3, 4, 1, 5, 8, 9, 2],
       [2, 8, 4, 6, 3, 9, 1, 5, 7],
       [6, 4, 1, 3, 5, 2, 9, 7, 8],
       [3, 2, 5, 7, 9, 8, 6, 1, 4],
       [8, 9, 7, 1, 6, 4, 3, 2, 5]])